In [ ]:
import xarray as xr

file_path = 'gcs-bucket/Output-2/december_23.nc'

ds = xr.open_dataset(file_path, decode_times=False)
ds

In [ ]:
import pandas as pd

ds = ds.assign_coords(
    time=pd.to_timedelta(ds.time.values, unit="ns")
)

print(ds.time)

In [ ]:
ds = ds[['total_precipitation_6hr']]

# Display the updated dataset to verify
print("Dataset after keeping only 'total_precipitation_6hr':")
print(ds)
ds

In [ ]:
import xarray as xr

original_timesteps = ds.sizes['time']
sum_interval = 4
new_timesteps = original_timesteps // sum_interval

summed_dataarrays = []

for i in range(new_timesteps):
    start_index = i * sum_interval
    end_index = start_index + sum_interval

    summed_interval_data = ds['total_precipitation_6hr'].isel(time=slice(start_index, end_index)).sum(dim='time')

    new_time_coord = ds.time.values[start_index]
    summed_interval_data = summed_interval_data.expand_dims(time=[new_time_coord])

    summed_dataarrays.append(summed_interval_data)

summed_ds = xr.concat(summed_dataarrays, dim='time')

ds_summed = xr.Dataset({'total_precipitation_6hr': summed_ds})

print("Summed Dataset Info:")
print(ds_summed)
ds_summed

In [ ]:
summed_df = ds_summed.to_dataframe()
summed_df = summed_df.reset_index()

summed_df['total_precipitation_6hr'] = summed_df['total_precipitation_6hr'] * 1000

In [ ]:
import pandas as pd

excel_file_path = 'station_coordinates.xlsx'
df_stations = pd.read_excel(excel_file_path)

print("Contents of the Excel file:")
display(df_stations.head())

In [ ]:
import numpy as np

summed_coords = summed_df[['lat', 'lon']].drop_duplicates().values

closest_coords_list = []

for index, station in df_stations.iterrows():
    station_lat = station['Latitude']
    station_lon = station['Longitude']

    distances_squared = (summed_coords[:, 0] - station_lat)**2 + (summed_coords[:, 1] - station_lon)**2

    closest_index = np.argmin(distances_squared)

    closest_coord = summed_coords[closest_index]

    closest_coords_list.append(tuple(closest_coord))

closest_coords_set = set(closest_coords_list)

mask = summed_df.apply(lambda row: (row['lat'], row['lon']) in closest_coords_set, axis=1)

filtered_summed_df = summed_df[mask].copy()

print("Filtered summed_df with closest coordinates:")
display(filtered_summed_df.head())

print(f"\nNumber of unique coordinate pairs in filtered_summed_df: {filtered_summed_df[['lat', 'lon']].drop_duplicates().shape[0]}")

In [ ]:
import pandas as pd

excel_filename = 'filtered_precipitation_by_day.xlsx'
with pd.ExcelWriter(excel_filename) as writer:
    df_day1 = filtered_summed_df[filtered_summed_df['time'] == pd.to_timedelta('0 days 06:00:00')]
    if not df_day1.empty:
        df_day1.to_excel(writer, sheet_name='1day', index=False)
        print("Data for Day 1 saved to '1day' sheet.")
    else:
        print("No data found for Day 1.")


    df_day2 = filtered_summed_df[filtered_summed_df['time'] == pd.to_timedelta('1 days 06:00:00')]
    if not df_day2.empty:
        df_day2.to_excel(writer, sheet_name='2day', index=False)
        print("Data for Day 2 saved to '2day' sheet.")
    else:
        print("No data found for Day 2.")

    df_day3 = filtered_summed_df[filtered_summed_df['time'] == pd.to_timedelta('2 days 06:00:00')]
    if not df_day3.empty:
        df_day3.to_excel(writer, sheet_name='3day', index=False)
        print("Data for Day 3 saved to '3day' sheet.")
    else:
        print("No data found for Day 3.")

print(f"\nFiltered data saved to {excel_filename}")

In [21]:
excel_file_path = 'filtered_precipitation_by_day.xlsx'
excel_file = pd.ExcelFile(excel_file_path)

In [ ]:
sheet_names = excel_file.sheet_names
modified_dfs = {}

for sheet_name in sheet_names:
    df = excel_file.parse(sheet_name)
    modified_dfs[sheet_name] = df

print("Sheets read into DataFrames:")
for name, df in modified_dfs.items():
    print(f"- '{name}': {df.shape} shape")

In [ ]:
start_dates = {
    '1day': '2023/12/02',
    '2day': '2023/12/03',
    '3day': '2023/12/04'
}

for sheet_name, df in modified_dfs.items():
    if 'time' in df.columns:
        df = df.drop(columns=['time'])

    start_date_str = start_dates[sheet_name]

    date_list = []

    for (lat, lon), group in df.groupby(['lat', 'lon']):
        group = group.sort_values(by='batch')
        for batch in group['batch']:
            current_date = pd.to_datetime(start_date_str) + pd.Timedelta(days=batch)
            formatted_date = current_date.strftime('%Y/%m/%d')
            date_list.append(formatted_date)

    original_columns_order = [col for col in df.columns if col != 'Date']
    df = df[original_columns_order].copy()
    df['Date'] = date_list 

    cols = ['Date'] + [col for col in df.columns if col != 'Date']
    df = df[cols]

    modified_dfs[sheet_name] = df

print("DataFrames modified with 'Date' column:")
for name, df in modified_dfs.items():
    print(f"- '{name}': {df.shape} shape, columns: {df.columns.tolist()}")
    display(df.head())

In [ ]:
excel_filename = 'december_23.xlsx'
with pd.ExcelWriter(excel_filename) as writer:
    for sheet_name, df in modified_dfs.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Updated Excel file saved to {excel_filename}")

In [ ]:
import pandas as pd
import glob
import os

folder = "/home/monad/pp-2"

# Find all Excel files
files = sorted(glob.glob(os.path.join(folder, "*.xlsx")))

merged = {"1day": [], "2day": [], "3day": []}

for f in files:
    for sheet in merged.keys():
        df = pd.read_excel(f, sheet_name=sheet)
        merged[sheet].append(df)

df_1day_all = pd.concat(merged["1day"]).sort_values("Date").reset_index(drop=True)
df_2day_all = pd.concat(merged["2day"]).sort_values("Date").reset_index(drop=True)
df_3day_all = pd.concat(merged["3day"]).sort_values("Date").reset_index(drop=True)

output_file = os.path.join(folder, "merged_forecasts.xlsx")
with pd.ExcelWriter(output_file) as writer:
    df_1day_all.to_excel(writer, sheet_name="1day", index=False)
    df_2day_all.to_excel(writer, sheet_name="2day", index=False)
    df_3day_all.to_excel(writer, sheet_name="3day", index=False)

print(f"Merged file saved at: {output_file}")

In [ ]:
import pandas as pd
import numpy as np

def compute_metrics_robust(df):
    records = []
    for station, g in df.groupby("Station", sort=False):
        valid = g[["total_precipitation_6hr", "BMD_Observed"]].dropna()
        n = len(valid)
        if n == 0:
            records.append({"Station": station, "N": 0, "CC": np.nan, "ME": np.nan, "RMSE": np.nan, "POD": np.nan})
            continue

        F = valid["total_precipitation_6hr"].to_numpy()
        O = valid["BMD_Observed"].to_numpy()

        if n >= 2 and np.std(F) > 0 and np.std(O) > 0:
            CC = np.corrcoef(F, O)[0, 1]
        else:
            CC = np.nan

        diff = F - O
        ME = float(np.mean(diff))
        RMSE = float(np.sqrt(np.mean(diff**2)))

        hits = int(np.sum((F > 0) & (O > 0)))
        misses = int(np.sum((F <= 0) & (O > 0)))
        POD = hits / (hits + misses) if (hits + misses) > 0 else np.nan

        records.append({"Station": station, "N": n, "CC": CC, "ME": ME, "RMSE": RMSE, "POD": POD})
    return pd.DataFrame.from_records(records)

file_path = "merged_forecasts_with_observed_rainfall_dateonly_formatted.xlsx"
xls = pd.ExcelFile(file_path)

outputs = {}
for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    for col in ["total_precipitation_6hr", "BMD_Observed"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    outputs[sheet] = compute_metrics_robust(df)

with pd.ExcelWriter("graphcast_station_metrics_robust.xlsx") as w:
    for sheet, out in outputs.items():
        out.to_excel(w, sheet_name=sheet, index=False)
print("Saved -> graphcast_station_metrics_robust.xlsx")

In [ ]:
import pandas as pd
import numpy as np

def compute_metrics_robust(df):
    records = []
    for station, g in df.groupby("Station", sort=False):
        valid = g[["ECMWF", "BMD_Observed"]].dropna()
        n = len(valid)
        if n == 0:
            records.append({"Station": station, "N": 0, "CC": np.nan, "ME": np.nan, "RMSE": np.nan, "POD": np.nan})
            continue

        F = valid["ECMWF"].to_numpy()
        O = valid["BMD_Observed"].to_numpy()

        if n >= 2 and np.std(F) > 0 and np.std(O) > 0:
            CC = np.corrcoef(F, O)[0, 1]
        else:
            CC = np.nan

        diff = F - O
        ME = float(np.mean(diff))
        RMSE = float(np.sqrt(np.mean(diff**2)))

        hits = int(np.sum((F > 0) & (O > 0)))
        misses = int(np.sum((F <= 0) & (O > 0)))
        POD = hits / (hits + misses) if (hits + misses) > 0 else np.nan

        records.append({"Station": station, "N": n, "CC": CC, "ME": ME, "RMSE": RMSE, "POD": POD})
    return pd.DataFrame.from_records(records)

file_path = "bmd_ecmwf_comparison.xlsx"
xls = pd.ExcelFile(file_path)

outputs = {}
for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    for col in ["ECMWF", "BMD_Observed"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    outputs[sheet] = compute_metrics_robust(df)

with pd.ExcelWriter("ecmwf_station_metrics_robust.xlsx") as w:
    for sheet, out in outputs.items():
        out.to_excel(w, sheet_name=sheet, index=False)
print("Saved -> ecmwf_station_metrics_robust.xlsx")

In [ ]:
import pandas as pd
import numpy as np

def compute_metrics_robust(df):
    records = []
    for station, g in df.groupby("Station", sort=False):
        valid = g[["ncep", "BMD_Observed"]].dropna()
        n = len(valid)
        if n == 0:
            records.append({"Station": station, "N": 0, "CC": np.nan, "ME": np.nan, "RMSE": np.nan, "POD": np.nan})
            continue

        F = valid["ncep"].to_numpy()
        O = valid["BMD_Observed"].to_numpy()

        if n >= 2 and np.std(F) > 0 and np.std(O) > 0:
            CC = np.corrcoef(F, O)[0, 1]
        else:
            CC = np.nan

        diff = F - O
        ME = float(np.mean(diff))
        RMSE = float(np.sqrt(np.mean(diff**2)))

        hits = int(np.sum((F > 0) & (O > 0)))
        misses = int(np.sum((F <= 0) & (O > 0)))
        POD = hits / (hits + misses) if (hits + misses) > 0 else np.nan

        records.append({"Station": station, "N": n, "CC": CC, "ME": ME, "RMSE": RMSE, "POD": POD})
    return pd.DataFrame.from_records(records)

file_path = "bmd_ncep_comparison.xlsx"
xls = pd.ExcelFile(file_path)

outputs = {}
for sheet in xls.sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)
    for col in ["ncep", "BMD_Observed"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    outputs[sheet] = compute_metrics_robust(df)

with pd.ExcelWriter("ncep_station_metrics_robust.xlsx") as w:
    for sheet, out in outputs.items():
        out.to_excel(w, sheet_name=sheet, index=False)
print("Saved -> ncep_station_metrics_robust.xlsx")